## UVCDAT

Ultrascale Visualization Climate Data Analysis Tools, or UVCDAT for short, is a python package particularly well suited for climate-data analysis. 

Its strengths are in 

* smart indexing to pull out particular chunks of geophysical data, 

* the built in commands that make manipulating the data easier, 

* ease by which data can be quickly uploaded and plotted. 

## Inputting geophysical data and plotting maps using UVCDAT

This notebook will walk us how to use UVCDAT to input and plot geophysical data using UVCDAT.

Let's first import our UVCDAT specific libraries

In [12]:
import cdms2     #For I/O
import cdutil    #For data manipulation
import genutil   #For even more data manipulation
import vcs       #UVCDAT's plotting package

In your directory, there should be a netcdf file with GPCP observations


In [13]:
GPCP_filename='pr_GPCP_197901-200909.nc'

Let's input that data using cdms2.open() and find out about what variables exist in the dataset

In [14]:
f_in=cdms2.open('pr_GPCP_197901-200909.nc')
f_in.listvariable()  #list the variables in the dataset

In [2]:
f_in.listdimension() #list the dimensions in the dataset

In [ ]:
# Try some other option yourself - there might be errors. How do we 

## Input and interogate data

'pr' is the data that we want - so let's simply load that

In [15]:
pr=f_in('pr')  #Here, the use of () loads the data. Only information of the variable is loaded if [] are used

In [3]:
pr.shape

What do the different dimensions pertain to?

In [4]:
pr.listdimnames()

In [5]:
pr.listattributes() #Query the other attributes to the data

You can interrogate what each one is by typing pr.attributename .

In [6]:
pr.units

You can just get the dimension data by using pr.get...

In [10]:
time=pr.getAxis(0)

In [7]:
time

In [8]:
time[0:10]

Now, it's a bit difficult to know what each of the times represent.
If only there were a way to get the dates for Jan 1 1995 to Dec 31 1999...

In [13]:
pr_gpcp_19951999=f_in('pr',time=('1995-01-01','1999-12-31'))

In [9]:
pr_gpcp_19951999.shape

You can do the same for lat and lon.** Try experimenting it with yourself. 

** You don't need to write lat and lon as strings. 

In [ ]:
# Try inputting data from a specific region

## Taking an average of the data

Now let's take the time-mean precipitation rate for 1995 to 1999. 

We make use of the averager function from cdutil

In [15]:
cdms2.setAutoBounds(1)  #Ensures that bounds to time and depth axes are generated if none exist
pr_gpcp_19951999_timemean=cdutil.averager(pr_gpcp_19951999,axis='t')  #For the other axes, use x or y

In [11]:
pr_gpcp_19951999_timemean.shape

Temporal Average Pt. 2

Another way of taking a time way will be to take a climatology of the YEAR. cdutil provides this capability.
You will notice that you can also take other climatologies, like the ANNUALCYCLE, which gives you a climatological mean for each month. 

In [18]:
pr_YEAR=cdutil.YEAR.climatology(pr)
pr_YEAR.shape

In [19]:
pr_ANNUALCYCLE=cdutil.ANNUALCYCLE.climatology(pr)
pr_ANNUALCYCLE.shape

## Plot the data

Now that we have the data, let's plot it.

The nice thing about vcs is that it does not take much to create a quick plot. At its simplest, it takes 2 lines.

In [10]:
x=vcs.init()
x.plot(pr_gpcp_19951999_timemean)

The default is a box plot (similar to pcolor). If we wanted an isofill plot, we can make things a bit more complicated.

In [3]:
y=vcs.init()
aa1=y.createisofill()
y.plot(pr_2_timemean,aa1)

In [ ]:
# Take the time mean of the region-specific data that you had created and plot it here! 

## Exercise: Input, average, and plot data

Let's say you have another dataset that has precipitation from a model. See the week3 directory for another dataset that looks like model data. 

Let's first input the data (for 1995-01-01 to 1999-12-31),

Then take the temporal average, 

and then plot the data below.

In [20]:
# Let's try this here

## Regridding data

You might have found that the data from this model is on a different grid as compared to the GPCP data. Now we'll go over regridding so that we can take the difference between the two plots.

We will be using ESMF regridding, which is what NCL uses for its regridding.
https://www.ncl.ucar.edu/Applications/ESMF.shtml

In [21]:
GPCP_grid=pr_gpcp_19951999_timemean.getGrid()
pr_NorESM_timemean_regrid=pr_NorESM_timemean.regrid(GPCP_grid,regrid_tool='esmf',regrid_method='conservative')

## Outputing data into a netcdf

Instead of having to recreate a regridded, climatological mean every single time, you might want to save the data as a netcdf file. You can use cdms2 to write the same data into a new netcdf file.
In the steps below, we not only save the netcdf file, but also the global attributes associated with the original netcdf file. 

In [ ]:
outfilename='pr_timeMean_GPCP_199501-199912.nc'
f_out=cdms2.open(outfilename,'w')
att_keys = f_in.attributes.keys()
att_dic = {}
for i in range(len(att_keys)):
    att_dic[i]=att_keys[i],f_in.attributes[att_keys[i]]
    to_out = att_dic[i]
    setattr(f_out,to_out[0],to_out[1])
pr_gprpc_19951999_timemean.id='pr'      #ensure that data manipulation hasn't changed the variable id
f_out.write(pr_gpcp_19951999_timemean)  #Write the output data
f_out.close()

## Plotting the difference map

Now, let's create a difference plot

Between the model and GPCP output. 

For other colorsmaps, consult here:
https://uvcdat.llnl.gov/vcs/Jupyter/Colormap_Tutorial.html

In [24]:
NorESM_minus_GPCP=pr_NorESM_timemean_regrid-pr_2_timemean
NorESM_minus_GPCP=NorESM_minus_GPCP*3600.*24.  #change units from kg m-2 s-1 to mm day-1 assuming 1 kg m-2 = 1 mm
NorESM_minus_GPCP.units='mm day-1'

z=vcs.init()
z.drawlogooff()      #turns off logo
aa2=z.createisofill()
Brown2bluegreen=vcs.colors.matplotlib2vcs('BrBG')
z.setcolormap(Brown2bluegreen)  #use UVCDAT's Brown to Blue Green colormap
levels=([-1e20,-4.,-3.,-2.,-1.,0.,1.,2.,3.,4.,1e20])
aa2.levels=levels
aa2.ext_1='y'   #Extension lower limit? 'y' or 'n'
aa2.ext_2='y'   #Extend upper limit?
z.plot(NorESM_minus_GPCP,aa2)

z.pdf('NorESM_minus_GPCP')


## Exercise: Take the difference between DJF and JJA in the GPCP and compare with NorESM1


In [ ]:
#For GPCP analysis and plot

In [ ]:
#For NorESM1 analysis and plot